In [1]:
import sys
import numpy as np
import datetime as dt
import pytz
import pandas as pd

sys.path.append('../')
from envir import config

In [2]:
noaa = pd.read_csv(config.dataFol+'noaa/2166257.csv')

/home/cusp/yg833/.conda/envs/birdsVE3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,9,11,13,15,17,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# add the lat long for washington square park
noaa['wsp_lat'] = 40.7311732
noaa['wsp_long'] = -73.9981389
# calc distance to wsp from weather station
noaa['distance'] = np.sqrt((noaa.LATITUDE-noaa.wsp_lat)**2 + (noaa.LONGITUDE-noaa.wsp_long)**2)
# clean up data frame
noaa.sort_values(by=['distance'], inplace=True)
noaa.drop_duplicates('DATE', inplace=True)
colnames = ['DATE', 'PRCP', 'SNOW', 'AWND', 'TMAX', 'TMIN']
noaa = noaa[colnames]
newcolnames = list(map(lambda x:x.lower(),colnames))
noaa.rename(columns=dict(zip(colnames, newcolnames)), inplace=True)
noaa['timestamp']=["{} {}".format(i, '00:00') for i in noaa['date'].tolist()]
noaa.drop(columns=['date'], inplace=True)

In [4]:
# weather by day data downloaded from NOAA
noaa.head()
#noaa.to_csv(config.dataFol+'clean_noaa/clean_noaa.csv', index=False)

,prcp,snow,awnd,tmax,tmin,timestamp
28173,0.00,0.0,NaN,NaN,NaN,2017-10-18 00:00
28392,0.35,0.0,NaN,NaN,NaN,2018-08-19 00:00
28393,0.06,0.0,NaN,NaN,NaN,2018-08-20 00:00
28394,0.00,0.0,NaN,NaN,NaN,2018-08-21 00:00
28395,0.39,0.0,NaN,NaN,NaN,2018-08-22 00:00


#### updated weather data from https://mesonet.agron.iastate.edu/request/download.phtml?network=NY_ASOS
station: [NYC] New York City
Dates: 2017/01/01 - 2020/06/09

In [5]:
weather = pd.read_hdf(config.dataFol+'spl/weather_df.h5')

In [6]:
weather['timestamp'] = pd.DatetimeIndex(weather.datetime).tz_convert('UTC')

In [7]:
weather['timestamp'] = weather.timestamp.astype('str').str[0:16]

In [8]:
weather = weather[['timestamp','temp_celcius', 'dewp_celcius', 'rh_percentage', 'wind_dir',
                   'wind_speed_mph', 'sea_level_pressure_mb', 'precipitation_mm',
                   'visibility_miles', 'gust_mph', 'peak_wind_gust_mph']]

In [9]:
weather.to_csv(config.dataFol+'clean_noaa/clean_weather.csv', index=False)